## 📋 **Project Overview**

**Dataset:** MIMIC-III (Medical Information Mart for Intensive Care III)
- Contains de-identified health data from ~60,000 ICU admissions
- Available through PhysioNet on Google Cloud BigQuery
- Requires special access credentials and authentication

**Methodology:** 
- **Temporal Design:** Extract features from first 48 hours, predict outcomes after 6-hour gap
- **Patient Selection:** First admission only, minimum 54-hour stay requirement
- **Feature Extraction:** Multiple clinical data modalities (demographics, vitals, labs, medications, microbiology)
- **Model Requirements:** Calibrated probability outputs for all three targets

**Key Constraints:**
- No data leakage between temporal windows
- Patient-level data splits to prevent information leakage
- Clinical interpretability and feature importance analysis required

This notebook handles the necessary setup for running the project in both Google Colab and a local environment.

# Project prereqs

this section is for the prereqs for the project.    

### 1. Import Libraries

In [21]:
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# A robust way to check if we are in Google Colab
try:
    from google.colab import auth
    from google.colab import data_table
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

### 2. Configure Google Cloud Project

In [22]:
project_id = 'deep-atlas-413309' # Yair Bigquery Project ID
# project_id = 'light-legend-457315-k3' # Lital Bigquery Project ID

### 3. Authenticate and Initialize Client

**🔑 Authentication Process:**

The MIMIC-III dataset requires proper authentication to access. This cell handles two scenarios:

**In Google Colab:**
- Uses `auth.authenticate_user()` which will prompt you to:
  1. Click on the authentication link
  2. Sign in with your Google account (must have MIMIC-III access)
  3. Copy the authorization code back to the notebook
- Enables enhanced dataframe formatting for better visualization

**Locally:** 
- Uses credentials from the Google Cloud SDK
- **Required setup:** You must run this command once in your terminal:
  ```bash
  gcloud auth application-default login
  ```
- This opens a browser window for one-time authentication setup
- Credentials are then stored locally for future use

**⚠️ Important Notes:**
- Your Google account must have been granted access to the MIMIC-III dataset
- Access requires completing the CITI training and signing the data use agreement
- If authentication fails, check that you have the correct project permissions

In [23]:
if IN_COLAB:
    print("Authenticating in Google Colab...")
    auth.authenticate_user()
    data_table.enable_dataframe_formatter()
else:
    print("Using local Google Cloud SDK credentials.")

client = bigquery.Client(project=project_id)
print(f"BigQuery client initialized for project: {client.project}")

Using local Google Cloud SDK credentials.
BigQuery client initialized for project: deep-atlas-413309


### 4. Display Database Schema (Optional)

**📊 Understanding the MIMIC-III Structure:**

This section queries the database metadata to show you all available tables and their columns. This is extremely helpful for:

- **Understanding data organization:** See how clinical data is structured
- **Planning feature extraction:** Identify which tables contain the data you need
- **Learning the schema:** Understand relationships between different data types

**Key Tables You'll See:**
- **`patients`:** Basic demographics (age, gender, date of birth/death)
- **`admissions`:** Hospital admission details (admit/discharge times, diagnoses)
- **`icustays`:** ICU-specific information (ICU admit/discharge, length of stay)  
- **`chartevents`:** Vital signs and monitoring data (heart rate, blood pressure, etc.)
- **`labevents`:** Laboratory test results (blood work, chemistry panels)
- **`prescriptions`:** Medication orders and administration
- **`microbiologyevents`:** Culture results and antibiotic sensitivity testing

**💡 Pro Tip:** Keep this schema reference handy when writing feature extraction queries later in the notebook!

In [24]:
# The project ID for the public MIMIC-III dataset
schema_project_id = "physionet-data"
schema_dataset_id = "mimiciii_clinical"

# Query the INFORMATION_SCHEMA to get table and column details
schema_query = f"""
    SELECT
        table_name,
        column_name,
        data_type
    FROM `{schema_project_id}.{schema_dataset_id}.INFORMATION_SCHEMA.COLUMNS`
    ORDER BY
        table_name,
        ordinal_position;
"""

# Run the query
schema_df = client.query(schema_query).to_dataframe()

# Process and print the schema
if schema_df.empty:
    print("Could not retrieve schema. The dataset might be empty or inaccessible.")
else:
    print("--- MIMIC-III Clinical Dataset Schema ---")
    current_table = ""
    for index, row in schema_df.iterrows():
        if row['table_name'] != current_table:
            current_table = row['table_name']
            print(f"\n-- Table: {current_table} --")
        print(f"    - {row['column_name']} ({row['data_type']})")
    print("\n--- End of Schema ---")

c:\Users\pickh\pythonProject\MLH\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


--- MIMIC-III Clinical Dataset Schema ---

-- Table: admissions --
    - ROW_ID (INT64)
    - SUBJECT_ID (INT64)
    - HADM_ID (INT64)
    - ADMITTIME (DATETIME)
    - DISCHTIME (DATETIME)
    - DEATHTIME (DATETIME)
    - ADMISSION_TYPE (STRING)
    - ADMISSION_LOCATION (STRING)
    - DISCHARGE_LOCATION (STRING)
    - INSURANCE (STRING)
    - LANGUAGE (STRING)
    - RELIGION (STRING)
    - MARITAL_STATUS (STRING)
    - ETHNICITY (STRING)
    - EDREGTIME (DATETIME)
    - EDOUTTIME (DATETIME)
    - DIAGNOSIS (STRING)
    - HOSPITAL_EXPIRE_FLAG (INT64)
    - HAS_CHARTEVENTS_DATA (INT64)

-- Table: callout --
    - ROW_ID (INT64)
    - SUBJECT_ID (INT64)
    - HADM_ID (INT64)
    - SUBMIT_WARDID (INT64)
    - SUBMIT_CAREUNIT (STRING)
    - CURR_WARDID (INT64)
    - CURR_CAREUNIT (STRING)
    - CALLOUT_WARDID (INT64)
    - CALLOUT_SERVICE (STRING)
    - REQUEST_TELE (INT64)
    - REQUEST_RESP (INT64)
    - REQUEST_CDIFF (INT64)
    - REQUEST_MRSA (INT64)
    - REQUEST_VRE (INT64)
    - CALL

# Project Framework

This section outlines the development of our prediction models, adhering to the project guidelines. We will implement a full pipeline, from data extraction to model evaluation.

### Constants and Configuration

**⏱️ Temporal Design Parameters:**

These constants define the critical timing constraints for our prediction task:

```
|<--- DATA_COLLECTION_HOURS --->|<-- GAP -->|<--- PREDICTION WINDOW --->|
|           48 hours            |  6 hours  |     Monitor outcomes      |
|                               |           |                           |
Admission -----------------> Feature -----> Prediction -----------------> Discharge
          Extract vitals,    Cutoff        Point
          labs, meds, etc.                 
```

**Why These Specific Values?**

- **48-hour collection window:** Captures the critical initial period when most clinical decisions are made
- **6-hour prediction gap:** Ensures we're truly predicting future outcomes, not just describing current state
- **54-hour minimum stay:** Ensures all patients have enough data for both feature extraction and outcome definition

**Clinical Rationale:**
- Early intervention is crucial in critical care
- Most ICU deaths occur within the first few days  
- Hospital systems need time to act on predictions (hence the gap)
- Prolonged stay threshold (7 days) represents significant resource utilization

In [14]:
# Project constants based on the guidelines
DATA_COLLECTION_HOURS = 48
PREDICTION_GAP_HOURS = 6
MIN_HOSPITALIZATION_HOURS = DATA_COLLECTION_HOURS + PREDICTION_GAP_HOURS  # 54 hours

## 1. Data Extraction

**🎯 Goal:** Extract and preprocess the required data modalities for our cohort.

**📊 Required Data Modalities (per project guidelines):**
- ✅ **Demographic features** (age, gender, ethnicity, insurance)
- ✅ **Vital signs** (heart rate, blood pressure, temperature, respiratory rate, SpO2)
- ✅ **Laboratory test results** (basic metabolic panel, complete blood count, liver function)
- ✅ **Medications** (antibiotics, vasopressors, sedatives, insulin)
- ✅ **Microbiology events** (culture results, organism identification, antibiotic sensitivity)

**🏥 Patient Selection Criteria:**
- **First hospital admission only** (to avoid repeated admissions bias)
- **Minimum 54 hours hospitalization** (48h feature extraction + 6h prediction gap)
- **Has chartevents data** (ensures ICU-level monitoring was available)
- **From provided initial cohort CSV** (ensures reproducible patient selection)

**⏱️ Temporal Constraints:**
All features will be extracted from the **first 48 hours** of admission to ensure:
- No future information leakage
- Clinically actionable timing (early in hospital course)
- Sufficient data for robust feature engineering

**🔄 Processing Pipeline:**
1. Load initial cohort subject IDs from CSV
2. Extract first admission demographics for each patient  
3. Filter by minimum stay requirements
4. Prepare for target definition and feature extraction

### 1.1 Load Initial Cohort

**📄 About the Initial Cohort CSV:**

The `initial_cohort.csv` file contains ~32,500 subject IDs that have been pre-selected for this project. This ensures:

- **Reproducible results:** Everyone works with the same patient population
- **Quality control:** Patients have been pre-screened for data completeness
- **Project scope:** Manageable dataset size for analysis and modeling

**🔍 What happens here:**
1. Load the CSV file containing subject IDs
2. Convert to a list for use in BigQuery parameterized queries
3. Error handling if the file is missing or corrupted
4. Display cohort size for verification

In [15]:
# Load the initial cohort from the provided CSV file
try:
    cohort_subject_ids_df = pd.read_csv('../data/initial_cohort.csv')
    subject_ids = cohort_subject_ids_df['subject_id'].tolist()
    print(f"Successfully loaded {len(subject_ids)} subject IDs from initial_cohort.csv")
except FileNotFoundError:
    print("Error: 'data/initial_cohort.csv' not found.")
    print("Please ensure the file exists and the path is correct.")
    subject_ids = []  # Set to empty list to prevent query errors


subject_ids[:10]

Successfully loaded 32513 subject IDs from initial_cohort.csv


[22392, 2847, 12056, 25600, 73125, 13429, 5023, 47109, 50434, 2920]

### 1.2 Extract Initial Cohort Demographics

**🏥 SQL Query Breakdown:**

This complex query performs several critical steps:

**Step 1: `admissions_for_cohort`**
- Joins `patients` and `admissions` tables
- Filters to only our cohort patients (from CSV)
- Ensures admissions have chartevents data (ICU-level monitoring)
- Extracts key demographics and admission details

**Step 2: `first_admissions`** 
- Uses `ROW_NUMBER()` to rank admissions by time for each patient
- Ensures we only get the **first admission** per patient
- Critical for avoiding bias from repeated admissions

**Step 3: `final_cohort`**
- Calculates length of stay in hours using `DATETIME_DIFF`
- Filters for minimum 54-hour stays (our temporal requirement)
- Produces final analysis-ready cohort

**🎯 Key Filtering Criteria:**
- `has_chartevents_data = 1`: Ensures ICU-level monitoring
- `rn = 1`: First admission only  
- `length_of_stay_hours >= 54`: Meets temporal requirements

**📊 Expected Output:**
- ~28,500 patients (some filtered out due to stay length)
- Columns: demographics, admission/discharge times, length of stay
- Ready for target definition and feature extraction

In [25]:
initial_cohort_query = '''
-- This query extracts the first admission for a specified list of patients,
-- ensuring their hospital stay meets a minimum duration.

-- Step 1: Filter admissions to only include the patients specified in the cohort
WITH admissions_for_cohort AS (
    SELECT
        p.subject_id,
        a.hadm_id,
        a.admittime,
        a.dischtime,
        a.deathtime,
        p.gender,
        p.dob,
        a.ethnicity,
        a.insurance
    FROM `physionet-data.mimiciii_clinical.patients` p
    INNER JOIN `physionet-data.mimiciii_clinical.admissions` a ON p.subject_id = a.subject_id
    WHERE
        -- Only include patients from our initial cohort list
        p.subject_id IN UNNEST(@subject_ids)
        -- And ensure the admission has associated chart events
        AND a.has_chartevents_data = 1
),

-- Step 2: Identify the first admission for each patient
first_admissions AS (
    SELECT
        *,
        -- Assign a row number to each admission, ordered by time
        ROW_NUMBER() OVER(PARTITION BY subject_id ORDER BY admittime) as rn
    FROM admissions_for_cohort
),

-- Step 3: Calculate length of stay and filter by the minimum required duration
final_cohort AS (
    SELECT
        *,
        -- Calculate the duration of the hospital stay in hours
        DATETIME_DIFF(dischtime, admittime, HOUR) as length_of_stay_hours
    FROM first_admissions
    WHERE
        -- We only want the very first admission
        rn = 1
)

-- Final Selection: Select all columns from the processed and filtered cohort
SELECT *
FROM final_cohort
WHERE
    -- Ensure the hospital stay is long enough for our analysis
    length_of_stay_hours >= @min_hospitalization_hours
'''

query_params = [
    bigquery.ArrayQueryParameter("subject_ids", "INT64", subject_ids),
    bigquery.ScalarQueryParameter("min_hospitalization_hours", "INT64", MIN_HOSPITALIZATION_HOURS),
]

job_config = bigquery.QueryJobConfig(
    query_parameters=query_params
)

# Only run the query if subject_ids were loaded successfully
if subject_ids:
    initial_cohort_df = client.query(initial_cohort_query, job_config=job_config).to_dataframe()
    print(f"Found {len(initial_cohort_df)} patients from the CSV who meet the criteria.")
    display(initial_cohort_df.head())
else:
    print("Query skipped because no subject IDs were loaded.")
    initial_cohort_df = pd.DataFrame() # Create an empty dataframe

Found 28552 patients from the CSV who meet the criteria.


,subject_id,hadm_id,admittime,dischtime,deathtime,gender,dob,ethnicity,insurance,rn,length_of_stay_hours
0,421,174459,2157-04-13 02:08:00,2157-04-22 18:50:00,NaT,M,2105-07-15,HISPANIC/LATINO - DOMINICAN,Government,1,232
1,3886,165159,2125-01-09 17:16:00,2125-01-20 12:00:00,NaT,F,2058-08-10,WHITE,Government,1,259
2,4776,112029,2122-12-15 20:47:00,2122-12-18 12:20:00,NaT,M,2082-05-06,BLACK/AFRICAN AMERICAN,Government,1,64
3,16940,193636,2120-01-28 16:15:00,2120-03-29 12:20:00,NaT,F,2120-01-28,OTHER,Government,1,1460
4,17343,123590,2155-09-07 06:45:00,2155-09-11 10:40:00,NaT,M,2155-09-07,BLACK/AFRICAN AMERICAN,Government,1,100


### 1.3 Readmission Detection Query

**Step 1: `all_admissions_for_cohort_patients`**
- Query ALL admissions for our cohort patients (not just the cohort admission)
- This gives us the complete admission history for each patient

**Step 2: `cohort_with_next_admission`**  
- For each cohort admission (first admission), find the next admission
- Use `LEFT JOIN` with condition `a.admittime > c.dischtime` 
- Use `MIN()` to get the earliest subsequent admission
- Results in `next_admittime` for readmission detection

**🎯 Key Insights:**
- Many patients have only one admission (no readmission)
- Some patients have multiple readmissions (we want the first one)
- The 30-day window is applied later in the target definition function

**🛡️ Data Integrity:**
- Uses parameterized queries to prevent SQL injection
- Handles temporary table creation and cleanup
- Robust error handling for BigQuery operations

In [26]:
# Query to find subsequent admissions for readmission detection
readmission_query = """
-- Get all admissions for patients in our cohort to detect readmissions
WITH all_admissions_for_cohort_patients AS (
    SELECT 
        a.subject_id,
        a.hadm_id,
        DATETIME(a.admittime) as admittime  -- Ensure DATETIME type
    FROM `physionet-data.mimiciii_clinical.admissions` a
    WHERE a.subject_id IN UNNEST(@subject_ids)
),
cohort_with_next_admission AS (
    SELECT 
        c.subject_id,
        c.hadm_id,
        DATETIME(c.dischtime) as dischtime,  -- Ensure DATETIME type
        MIN(a.admittime) as next_admittime
    FROM initial_cohort_df c
    LEFT JOIN all_admissions_for_cohort_patients a 
        ON c.subject_id = a.subject_id 
        AND a.admittime > DATETIME(c.dischtime)  -- Cast both sides to DATETIME
    GROUP BY c.subject_id, c.hadm_id, c.dischtime
)
SELECT 
    subject_id,
    hadm_id,
    next_admittime
FROM cohort_with_next_admission
"""

# Execute the query only if we don't already have readmission data
if not initial_cohort_df.empty and subject_ids:
    # Create a temporary table from the initial_cohort_df to use in the query
    client.create_dataset('temp_dataset', exists_ok=True)
    table_ref = client.dataset('temp_dataset').table('initial_cohort_df')
    job = client.load_table_from_dataframe(
        initial_cohort_df[['subject_id', 'hadm_id', 'dischtime']], 
        table_ref
    )
    job.result()  # Wait for the table to be created
    
    # Set up query parameters
    query_params = [bigquery.ArrayQueryParameter("subject_ids", "INT64", subject_ids)]
    job_config = bigquery.QueryJobConfig(query_parameters=query_params)
    
    # Run the readmission query
    readmission_df = client.query(
        readmission_query.replace('initial_cohort_df', '`temp_dataset.initial_cohort_df`'),
        job_config=job_config
    ).to_dataframe()
    
    # Merge the readmission time back into our main cohort dataframe
    initial_cohort_df = pd.merge(
        initial_cohort_df, 
        readmission_df[['subject_id', 'hadm_id', 'next_admittime']], 
        on=['subject_id', 'hadm_id'], 
        how='left'
    )
    
    # Clean up the temp table
    client.delete_table(table_ref, not_found_ok=True)
    print(f"Added readmission data for {len(initial_cohort_df)} patients")
    display(initial_cohort_df.head())
else:
    print("Skipped readmission query - no cohort data available")

c:\Users\pickh\pythonProject\MLH\.venv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(
c:\Users\pickh\pythonProject\MLH\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Added readmission data for 28552 patients


,subject_id,hadm_id,admittime,dischtime,deathtime,gender,dob,ethnicity,insurance,rn,length_of_stay_hours,next_admittime
0,421,174459,2157-04-13 02:08:00,2157-04-22 18:50:00,NaT,M,2105-07-15,HISPANIC/LATINO - DOMINICAN,Government,1,232,NaT
1,3886,165159,2125-01-09 17:16:00,2125-01-20 12:00:00,NaT,F,2058-08-10,WHITE,Government,1,259,NaT
2,4776,112029,2122-12-15 20:47:00,2122-12-18 12:20:00,NaT,M,2082-05-06,BLACK/AFRICAN AMERICAN,Government,1,64,NaT
3,16940,193636,2120-01-28 16:15:00,2120-03-29 12:20:00,NaT,F,2120-01-28,OTHER,Government,1,1460,NaT
4,17343,123590,2155-09-07 06:45:00,2155-09-11 10:40:00,NaT,M,2155-09-07,BLACK/AFRICAN AMERICAN,Government,1,100,NaT


### 1.4 Feature Extraction

**🎯 Goal:** Extract the required clinical data modalities from MIMIC-III tables.

**📊 Data Modalities to Extract:**
- **Vital signs** (heart rate, blood pressure, temperature, respiratory rate, SpO2)
- **Laboratory test results** (basic metabolic panel, complete blood count, liver function)
- **Medications** (antibiotics, vasopressors, sedatives, insulin)
- **Microbiology events** (culture results, organism identification, antibiotic sensitivity)

**⏱️ Temporal Constraint:** All features extracted from **first 48 hours** of admission only.

#### 1.4.1 Vital Signs Extraction

**🫀 Extracting Vital Signs from Chartevents:**

Critical vital signs for ICU prediction models.

In [ ]:
# Vital signs ITEMID mapping for MIMIC-III
vital_signs_items = {
    'heart_rate': [211, 220045],  # Heart Rate
    'sbp': [51, 442, 455, 6701, 220179, 220050],  # Systolic BP
    'dbp': [8368, 8440, 8441, 8555, 220180, 220051],  # Diastolic BP
    'mbp': [52, 6702, 443, 220052, 220181, 225312],  # Mean BP
    'resp_rate': [615, 618, 220210, 224690],  # Respiratory Rate
    'temperature': [223762, 676, 223761, 678],  # Temperature
    'spo2': [646, 220277],  # SpO2
    'glucose': [807, 811, 1529, 3745, 3744, 225664, 220621, 226537]  # Glucose
}

# Extract vital signs within 48 hours of admission
vital_signs_query = """
WITH cohort_admissions AS (
    SELECT subject_id, hadm_id, DATETIME(admittime) as admittime
    FROM `temp_dataset.initial_cohort_df`
),
vital_signs_raw AS (
    SELECT 
        c.subject_id,
        c.hadm_id,
        ce.itemid,
        DATETIME(ce.charttime) as charttime,
        ce.valuenum,
        CASE 
            WHEN ce.itemid IN UNNEST(@heart_rate_items) THEN 'heart_rate'
            WHEN ce.itemid IN UNNEST(@sbp_items) THEN 'sbp'
            WHEN ce.itemid IN UNNEST(@dbp_items) THEN 'dbp'
            WHEN ce.itemid IN UNNEST(@mbp_items) THEN 'mbp'
            WHEN ce.itemid IN UNNEST(@resp_rate_items) THEN 'resp_rate'
            WHEN ce.itemid IN UNNEST(@temperature_items) THEN 'temperature'
            WHEN ce.itemid IN UNNEST(@spo2_items) THEN 'spo2'
            WHEN ce.itemid IN UNNEST(@glucose_items) THEN 'glucose'
        END AS vital_type
    FROM cohort_admissions c
    INNER JOIN `physionet-data.mimiciii_clinical.chartevents` ce
        ON c.subject_id = ce.subject_id 
        AND c.hadm_id = ce.hadm_id
    WHERE 
        ce.itemid IN UNNEST(@all_vital_items)
        AND DATETIME(ce.charttime) >= c.admittime
        AND DATETIME(ce.charttime) <= DATETIME_ADD(c.admittime, INTERVAL @data_collection_hours HOUR)
        AND ce.valuenum IS NOT NULL
        AND ce.error != 1
)
SELECT 
    subject_id,
    hadm_id,
    vital_type,
    COUNT(*) as measurement_count,
    AVG(valuenum) as avg_value,
    MIN(valuenum) as min_value,
    MAX(valuenum) as max_value,
    STDDEV(valuenum) as std_value
FROM vital_signs_raw
GROUP BY subject_id, hadm_id, vital_type
"""

if not initial_cohort_df.empty:
    # Create temporary table
    client.create_dataset('temp_dataset', exists_ok=True)
    table_ref = client.dataset('temp_dataset').table('initial_cohort_df')
    job = client.load_table_from_dataframe(
        initial_cohort_df[['subject_id', 'hadm_id', 'admittime']], 
        table_ref
    )
    job.result()
    
    # Prepare all vital sign item IDs
    all_vital_items = []
    for items in vital_signs_items.values():
        all_vital_items.extend(items)
    
    # Set up query parameters
    query_params = [
        bigquery.ArrayQueryParameter("heart_rate_items", "INT64", vital_signs_items['heart_rate']),
        bigquery.ArrayQueryParameter("sbp_items", "INT64", vital_signs_items['sbp']),
        bigquery.ArrayQueryParameter("dbp_items", "INT64", vital_signs_items['dbp']),
        bigquery.ArrayQueryParameter("mbp_items", "INT64", vital_signs_items['mbp']),
        bigquery.ArrayQueryParameter("resp_rate_items", "INT64", vital_signs_items['resp_rate']),
        bigquery.ArrayQueryParameter("temperature_items", "INT64", vital_signs_items['temperature']),
        bigquery.ArrayQueryParameter("spo2_items", "INT64", vital_signs_items['spo2']),
        bigquery.ArrayQueryParameter("glucose_items", "INT64", vital_signs_items['glucose']),
        bigquery.ArrayQueryParameter("all_vital_items", "INT64", all_vital_items),
        bigquery.ScalarQueryParameter("data_collection_hours", "INT64", DATA_COLLECTION_HOURS),
    ]
    
    job_config = bigquery.QueryJobConfig(query_parameters=query_params)
    
    # Execute vital signs query
    vital_signs_df = client.query(vital_signs_query, job_config=job_config).to_dataframe()
    
    print(f"Extracted vital signs for {vital_signs_df['subject_id'].nunique()} patients")
    print(f"Vital signs extracted: {vital_signs_df['vital_type'].unique()}")
    display(vital_signs_df.head(10))
else:
    print("No cohort data available for vital signs extraction")
    vital_signs_df = pd.DataFrame()

#### 1.4.2 Laboratory Results Extraction

**🧪 Extracting Lab Results from Labevents (ONLY):**

**⚠️ Important:** Using ONLY labevents table as per project instructions.

In [ ]:
# Laboratory test ITEMID mapping for MIMIC-III (labevents only)
lab_items = {
    # Basic Metabolic Panel
    'sodium': [50983],
    'potassium': [50971], 
    'chloride': [50902],
    'co2': [50882],
    'bun': [51006],
    'creatinine': [50912],
    'glucose': [50931],
    
    # Complete Blood Count
    'hemoglobin': [51222],
    'hematocrit': [51221], 
    'wbc': [51301],
    'platelet': [51265],
    
    # Liver Function
    'bilirubin_total': [50885],
    'alt': [50861],
    'ast': [50878],
    'alkaline_phosphatase': [50863],
    
    # Additional important labs
    'lactate': [50813],
    'ph': [50820],
    'albumin': [50862]
}

# Extract lab results within 48 hours of admission
lab_results_query = """
WITH cohort_admissions AS (
    SELECT subject_id, hadm_id, admittime
    FROM `temp_dataset.initial_cohort_df`
),
lab_results_raw AS (
    SELECT 
        c.subject_id,
        c.hadm_id,
        le.itemid,
        le.charttime,
        le.valuenum,
        CASE 
            WHEN le.itemid IN UNNEST(@sodium_items) THEN 'sodium'
            WHEN le.itemid IN UNNEST(@potassium_items) THEN 'potassium'
            WHEN le.itemid IN UNNEST(@chloride_items) THEN 'chloride'
            WHEN le.itemid IN UNNEST(@co2_items) THEN 'co2'
            WHEN le.itemid IN UNNEST(@bun_items) THEN 'bun'
            WHEN le.itemid IN UNNEST(@creatinine_items) THEN 'creatinine'
            WHEN le.itemid IN UNNEST(@glucose_items) THEN 'glucose_lab'
            WHEN le.itemid IN UNNEST(@hemoglobin_items) THEN 'hemoglobin'
            WHEN le.itemid IN UNNEST(@hematocrit_items) THEN 'hematocrit'
            WHEN le.itemid IN UNNEST(@wbc_items) THEN 'wbc'
            WHEN le.itemid IN UNNEST(@platelet_items) THEN 'platelet'
            WHEN le.itemid IN UNNEST(@bilirubin_items) THEN 'bilirubin_total'
            WHEN le.itemid IN UNNEST(@alt_items) THEN 'alt'
            WHEN le.itemid IN UNNEST(@ast_items) THEN 'ast'
            WHEN le.itemid IN UNNEST(@alkphos_items) THEN 'alkaline_phosphatase'
            WHEN le.itemid IN UNNEST(@lactate_items) THEN 'lactate'
            WHEN le.itemid IN UNNEST(@ph_items) THEN 'ph'
            WHEN le.itemid IN UNNEST(@albumin_items) THEN 'albumin'
        END AS lab_type
    FROM cohort_admissions c
    INNER JOIN `physionet-data.mimiciii_clinical.labevents` le
        ON c.subject_id = le.subject_id 
        AND c.hadm_id = le.hadm_id
    WHERE 
        le.itemid IN UNNEST(@all_lab_items)
        AND le.charttime >= c.admittime
        AND le.charttime <= DATETIME_ADD(c.admittime, INTERVAL @data_collection_hours HOUR)
        AND le.valuenum IS NOT NULL
)
SELECT 
    subject_id,
    hadm_id,
    lab_type,
    COUNT(*) as measurement_count,
    AVG(valuenum) as avg_value,
    MIN(valuenum) as min_value,
    MAX(valuenum) as max_value,
    STDDEV(valuenum) as std_value
FROM lab_results_raw
GROUP BY subject_id, hadm_id, lab_type
"""

if not initial_cohort_df.empty:
    # Prepare all lab item IDs
    all_lab_items = []
    for items in lab_items.values():
        all_lab_items.extend(items)
    
    # Set up query parameters
    query_params = [
        bigquery.ArrayQueryParameter("sodium_items", "INT64", lab_items['sodium']),
        bigquery.ArrayQueryParameter("potassium_items", "INT64", lab_items['potassium']),
        bigquery.ArrayQueryParameter("chloride_items", "INT64", lab_items['chloride']),
        bigquery.ArrayQueryParameter("co2_items", "INT64", lab_items['co2']),
        bigquery.ArrayQueryParameter("bun_items", "INT64", lab_items['bun']),
        bigquery.ArrayQueryParameter("creatinine_items", "INT64", lab_items['creatinine']),
        bigquery.ArrayQueryParameter("glucose_items", "INT64", lab_items['glucose']),
        bigquery.ArrayQueryParameter("hemoglobin_items", "INT64", lab_items['hemoglobin']),
        bigquery.ArrayQueryParameter("hematocrit_items", "INT64", lab_items['hematocrit']),
        bigquery.ArrayQueryParameter("wbc_items", "INT64", lab_items['wbc']),
        bigquery.ArrayQueryParameter("platelet_items", "INT64", lab_items['platelet']),
        bigquery.ArrayQueryParameter("bilirubin_items", "INT64", lab_items['bilirubin_total']),
        bigquery.ArrayQueryParameter("alt_items", "INT64", lab_items['alt']),
        bigquery.ArrayQueryParameter("ast_items", "INT64", lab_items['ast']),
        bigquery.ArrayQueryParameter("alkphos_items", "INT64", lab_items['alkaline_phosphatase']),
        bigquery.ArrayQueryParameter("lactate_items", "INT64", lab_items['lactate']),
        bigquery.ArrayQueryParameter("ph_items", "INT64", lab_items['ph']),
        bigquery.ArrayQueryParameter("albumin_items", "INT64", lab_items['albumin']),
        bigquery.ArrayQueryParameter("all_lab_items", "INT64", all_lab_items),
        bigquery.ScalarQueryParameter("data_collection_hours", "INT64", DATA_COLLECTION_HOURS),
    ]
    
    job_config = bigquery.QueryJobConfig(query_parameters=query_params)
    
    # Execute lab results query
    lab_results_df = client.query(lab_results_query, job_config=job_config).to_dataframe()
    
    print(f"Extracted lab results for {lab_results_df['subject_id'].nunique()} patients")
    print(f"Lab tests extracted: {lab_results_df['lab_type'].unique()}")
    display(lab_results_df.head(10))
else:
    print("No cohort data available for lab results extraction")
    lab_results_df = pd.DataFrame()

#### 1.4.3 Medications Extraction

**💊 Extracting Medications from Prescriptions:**

Focus on clinically important medication categories.

In [ ]:
# Medication categories of clinical interest
medication_keywords = {
    'antibiotics': ['antibiotic', 'penicillin', 'cephalexin', 'amoxicillin', 'ciprofloxacin', 
                   'vancomycin', 'gentamicin', 'ceftriaxone', 'azithromycin', 'clindamycin'],
    'vasopressors': ['norepinephrine', 'epinephrine', 'dopamine', 'dobutamine', 'vasopressin', 
                    'phenylephrine', 'milrinone'],
    'sedatives': ['propofol', 'midazolam', 'lorazepam', 'fentanyl', 'morphine', 'dexmedetomidine'],
    'insulin': ['insulin', 'humulin', 'novolog', 'lantus', 'humalog'],
    'anticoagulants': ['heparin', 'warfarin', 'enoxaparin', 'fondaparinux'],
    'diuretics': ['furosemide', 'lasix', 'hydrochlorothiazide', 'spironolactone'],
    'beta_blockers': ['metoprolol', 'propranolol', 'atenolol', 'carvedilol'],
    'ace_inhibitors': ['lisinopril', 'enalapril', 'captopril', 'ramipril']
}

# Extract medications within 48 hours of admission
medications_query = """
WITH cohort_admissions AS (
    SELECT subject_id, hadm_id, admittime
    FROM `temp_dataset.initial_cohort_df`
),
medications_raw AS (
    SELECT 
        c.subject_id,
        c.hadm_id,
        p.drug,
        p.drug_name_generic,
        p.startdate,
        CASE 
            WHEN REGEXP_CONTAINS(LOWER(COALESCE(p.drug, p.drug_name_generic, '')), r'antibiotic|penicillin|cephalexin|amoxicillin|ciprofloxacin|vancomycin|gentamicin|ceftriaxone|azithromycin|clindamycin') THEN 'antibiotics'
            WHEN REGEXP_CONTAINS(LOWER(COALESCE(p.drug, p.drug_name_generic, '')), r'norepinephrine|epinephrine|dopamine|dobutamine|vasopressin|phenylephrine|milrinone') THEN 'vasopressors'
            WHEN REGEXP_CONTAINS(LOWER(COALESCE(p.drug, p.drug_name_generic, '')), r'propofol|midazolam|lorazepam|fentanyl|morphine|dexmedetomidine') THEN 'sedatives'
            WHEN REGEXP_CONTAINS(LOWER(COALESCE(p.drug, p.drug_name_generic, '')), r'insulin|humulin|novolog|lantus|humalog') THEN 'insulin'
            WHEN REGEXP_CONTAINS(LOWER(COALESCE(p.drug, p.drug_name_generic, '')), r'heparin|warfarin|enoxaparin|fondaparinux') THEN 'anticoagulants'
            WHEN REGEXP_CONTAINS(LOWER(COALESCE(p.drug, p.drug_name_generic, '')), r'furosemide|lasix|hydrochlorothiazide|spironolactone') THEN 'diuretics'
            WHEN REGEXP_CONTAINS(LOWER(COALESCE(p.drug, p.drug_name_generic, '')), r'metoprolol|propranolol|atenolol|carvedilol') THEN 'beta_blockers'
            WHEN REGEXP_CONTAINS(LOWER(COALESCE(p.drug, p.drug_name_generic, '')), r'lisinopril|enalapril|captopril|ramipril') THEN 'ace_inhibitors'
        END AS medication_category
    FROM cohort_admissions c
    INNER JOIN `physionet-data.mimiciii_clinical.prescriptions` p
        ON c.subject_id = p.subject_id 
        AND c.hadm_id = p.hadm_id
    WHERE 
        p.startdate >= DATE(c.admittime)
        AND p.startdate <= DATE_ADD(DATE(c.admittime), INTERVAL @data_collection_days DAY)
        AND (p.drug IS NOT NULL OR p.drug_name_generic IS NOT NULL)
)
SELECT 
    subject_id,
    hadm_id,
    medication_category,
    COUNT(*) as prescription_count,
    COUNT(DISTINCT drug) as unique_drugs
FROM medications_raw
WHERE medication_category IS NOT NULL
GROUP BY subject_id, hadm_id, medication_category
"""

if not initial_cohort_df.empty:
    # Set up query parameters
    query_params = [
        bigquery.ScalarQueryParameter("data_collection_days", "INT64", 2),  # 48 hours = 2 days
    ]
    
    job_config = bigquery.QueryJobConfig(query_parameters=query_params)
    
    # Execute medications query
    medications_df = client.query(medications_query, job_config=job_config).to_dataframe()
    
    print(f"Extracted medications for {medications_df['subject_id'].nunique()} patients")
    print(f"Medication categories extracted: {medications_df['medication_category'].unique()}")
    display(medications_df.head(10))
else:
    print("No cohort data available for medications extraction")
    medications_df = pd.DataFrame()

#### 1.4.4 Microbiology Events Extraction

**🦠 Extracting Microbiology from Microbiologyevents:**

Culture results and organism identification within 48 hours.

In [ ]:
# Extract microbiology events within 48 hours of admission
microbiology_query = """
WITH cohort_admissions AS (
    SELECT subject_id, hadm_id, admittime
    FROM `temp_dataset.initial_cohort_df`
),
microbiology_raw AS (
    SELECT 
        c.subject_id,
        c.hadm_id,
        me.chartdate,
        me.spec_type_desc,
        me.org_name,
        me.interpretation,
        CASE 
            WHEN me.spec_type_desc IS NOT NULL THEN 'culture_ordered'
            ELSE NULL
        END AS culture_type,
        CASE 
            WHEN me.org_name IS NOT NULL AND me.org_name != '' THEN 'organism_found'
            ELSE 'no_growth'
        END AS culture_result,
        CASE 
            WHEN LOWER(me.interpretation) LIKE '%sensitive%' OR LOWER(me.interpretation) LIKE '%susceptible%' THEN 'sensitive'
            WHEN LOWER(me.interpretation) LIKE '%resistant%' THEN 'resistant'
            WHEN LOWER(me.interpretation) LIKE '%intermediate%' THEN 'intermediate'
            ELSE 'unknown'
        END AS antibiotic_sensitivity
    FROM cohort_admissions c
    INNER JOIN `physionet-data.mimiciii_clinical.microbiologyevents` me
        ON c.subject_id = me.subject_id 
        AND c.hadm_id = me.hadm_id
    WHERE 
        me.chartdate >= DATE(c.admittime)
        AND me.chartdate <= DATE_ADD(DATE(c.admittime), INTERVAL @data_collection_days DAY)
)
SELECT 
    subject_id,
    hadm_id,
    'cultures_ordered' as micro_type,
    COUNT(*) as count_value
FROM microbiology_raw
GROUP BY subject_id, hadm_id

UNION ALL

SELECT 
    subject_id,
    hadm_id,
    culture_result as micro_type,
    COUNT(*) as count_value
FROM microbiology_raw
GROUP BY subject_id, hadm_id, culture_result

UNION ALL

SELECT 
    subject_id,
    hadm_id,
    CONCAT('sensitivity_', antibiotic_sensitivity) as micro_type,
    COUNT(*) as count_value
FROM microbiology_raw
WHERE antibiotic_sensitivity != 'unknown'
GROUP BY subject_id, hadm_id, antibiotic_sensitivity
"""

if not initial_cohort_df.empty:
    # Set up query parameters
    query_params = [
        bigquery.ScalarQueryParameter("data_collection_days", "INT64", 2),  # 48 hours = 2 days
    ]
    
    job_config = bigquery.QueryJobConfig(query_parameters=query_params)
    
    # Execute microbiology query
    microbiology_df = client.query(microbiology_query, job_config=job_config).to_dataframe()
    
    print(f"Extracted microbiology data for {microbiology_df['subject_id'].nunique()} patients")
    print(f"Microbiology types extracted: {microbiology_df['micro_type'].unique()}")
    display(microbiology_df.head(10))
else:
    print("No cohort data available for microbiology extraction")
    microbiology_df = pd.DataFrame()

### 1.5 Exploratory Data Analysis

**🎯 Goal:** Understand the characteristics of our extracted patient cohort and clinical features.

This section analyzes the basic demographic characteristics and data quality of our ICU cohort to inform target definition and modeling decisions.

**📊 What We'll Explore:**
- Demographics (age, gender, ethnicity, insurance)
- Length of stay patterns 
- Missing data assessment
- Data quality validation

**⚠️ Important Note:** This EDA happens AFTER feature extraction to understand our complete dataset.

In [ ]:
#### 1.5.1 Demographic Analysis

**👥 Understanding Our Patient Population:**

This analysis examines the basic demographic characteristics of our ICU cohort after feature extraction.

## 3. Data Partition

**🎯 Goal:** Split the data into training, validation, and test sets.

To prevent data leakage, we will split our cohort by `subject_id` before any feature engineering or preprocessing that looks across patients. A standard 60/20/20 split will be used.

In [ ]:
#### 1.5.2 Hospital Stay Analysis

**🏥 Length of Stay Patterns:**

Understanding length of stay patterns before defining the prolonged stay target.